In [1]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import load_model
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
def load_and_preprocess_data(preprocess_images=True):
    base_dir = r"D:\Users\RESA\Coding\Alzheimer-Classification-with-CNN-SVM\Notebook\Preprocessing\image_selected.npz"
    
    # Load data from the NPZ file
    loaded_data = np.load(base_dir, allow_pickle=True)
    loaded_combined_slices = loaded_data[list(loaded_data.keys())[0]]

    # Prepare lists for input (X) and output (Y) data
    X = []
    Y = []

    # Create a label mapping for each class
    label_mapping = {'AD': 0, 'CN': 1, 'EMCI': 2, 'LMCI': 3}

    # Iterate through the loaded data
    for label, plane_slices in loaded_combined_slices.item().items():
        for plane, slices in plane_slices.items():
            for selected_slice in slices:
                position, original_slice = selected_slice

                if preprocess_images:
                    # Expand dimensions and repeat for RGB representation
                    original_slice = np.expand_dims(original_slice, axis=-1)
                    original_slice = np.repeat(original_slice, 3, axis=-1)

                    # Min-Max Scaling to bring pixel values to the range [0, 1]
                    min_val = np.min(original_slice)
                    max_val = np.max(original_slice)
                    original_slice = (original_slice - min_val) / (max_val - min_val)

                X.append(original_slice)
                Y.append(label_mapping[label])

    # Convert lists to NumPy arrays
    X = np.array(X) 
    Y = np.array(Y)

    return X, Y

In [3]:
X, Y = load_and_preprocess_data()

In [4]:
print("X shape:", X.shape)
print("Y shape:", Y.shape)

X shape: (7200, 224, 224, 3)
Y shape: (7200,)


In [5]:
from keras import backend as K

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

In [6]:

class_names = ["AD", "CN", "EMCI", "LMCI"]

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [8]:
model_paths = [
    r"D:\Users\RESA\Coding\Model\densenet121_svm_model_poly.pkl",
    r"D:\Users\RESA\Coding\Model\densenet121_svm_model_rbf.pkl",
    r"D:\Users\RESA\Coding\Model\mobilenetv2_svm_model_poly.pkl",
    r"D:\Users\RESA\Coding\Model\mobilenetv2_svm_model_rbf.pkl",
    r"D:\Users\RESA\Coding\Model\resnet50_svm_model_poly.pkl",
    r"D:\Users\RESA\Coding\Model\resnet50_svm_model_rbf.pkl",
    r"D:\Users\RESA\Coding\Model\VGG16_svm_model_poly.pkl",
    r"D:\Users\RESA\Coding\Model\VGG16_svm_model_rbf.pkl",
]

In [9]:
models = [pickle.load(open(model_path, 'rb')) for model_path in model_paths]

In [10]:
label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(Y_test)

In [11]:
accuracies = []
conf_matrices = [] 

models_names = [model_path.split('\\')[-1].split('_')[0] for model_path in model_paths]

In [12]:
for model in models:
    y_pred = model.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test_encoded, y_pred)
    accuracies.append(accuracy)
    
    # Calculate confusion matrix
    cm = confusion_matrix(y_test_encoded, y_pred)
    conf_matrices.append(cm)

ValueError: Found array with dim 4. SVC expected <= 2.

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(models_names, accuracies, color='skyblue')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Accuracy Comparison for Different Models')
plt.ylim(0, 1) 
plt.show()

In [ ]:
for i, model_name in enumerate(models_names):
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrices[i], display_labels=label_encoder.classes_)
    disp.plot(cmap=plt.cm.Blues, values_format=".4g")
    plt.title(f'Confusion Matrix for {model_name}')
    plt.show()